<h2>Agentic RAG App: Groq LLM</h2>

<h3>LLM

In [1]:
# LangChain has a Groq chat model integration:
from langchain_groq import ChatGroq

/Users/cameroncochrane/Data Projects/Agentic-RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the API groq API key:
from pathlib import Path
# Bespoke API key loader functions:
from load_keys import *
import os

In [3]:
p = Path("keys/groq.json")
groq_api_key = load_groq_key(p)
os.environ['GROQ_API_KEY'] = groq_api_key

In [4]:
# Initialise the model:
model = ChatGroq(
            model="openai/gpt-oss-120b",
            temperature=0.2,
            max_retries=2,
        )

In [5]:
initial_message = [("human","hello, how are you?")]
answer_1 = model.invoke(initial_message) # .invoke to send a message to the model.

In [6]:
extracted_message = answer_1.content # x.content returns the message for the user.
print(extracted_message)

Hello! I'm doing great, thank you for asking. How can I assist you today?


In [7]:
internet_message = [("human","Do you have internet access?")]
internet_answer = model.invoke(internet_message)
print(internet_answer.content)

I don’t have the ability to browse the web or access real‑time internet data. My responses are based on the information I was trained on, which goes up to September 2021 (with some updates through 2024). If you need the most current information, you’ll need to check a reliable source directly.


In [8]:
def load_llm(model_type:str = 'openai/gpt-oss-120b',api_key_path:str = "keys/groq.json"):
    
    p = Path(api_key_path)

    groq_api_key = load_groq_key(p)
    os.environ['GROQ_API_KEY'] = groq_api_key
    
    model = ChatGroq(
            model=model_type,
            temperature=0.2,
            max_retries=2,
        )
    
    return model

In [9]:
def give_message(model,message):

    full_answer = model.invoke(message)
    extracted_answer = full_answer.content

    return extracted_answer

In [10]:
model_1 = load_llm()
extracted_answer = give_message(model_1,"What are some of your use cases?")
print(extracted_answer)

I’m a versatile, text‑based AI, so I can help with a wide range of tasks across many domains. Here are some of the most common use cases people find valuable:

| Category | Typical Tasks | How I Help |
|----------|----------------|------------|
| **Writing & Editing** | • Drafting emails, reports, essays, blog posts, social‑media captions<br>• Proofreading for grammar, style, tone, and clarity<br>• Re‑phrasing, summarizing, or expanding text | I generate first drafts, suggest improvements, and polish language to match the desired audience or brand voice. |
| **Creative Work** | • Brainstorming story ideas, plot twists, characters, world‑building<br>• Writing poetry, song lyrics, jokes, or marketing copy<br>• Generating prompts for visual art or DALL‑E | I provide fresh concepts, iterate on feedback, and help you overcome writer’s block. |
| **Learning & Tutoring** | • Explaining concepts in math, science, history, programming, etc.<br>• Walking through problem‑solving steps or proofs<b

In [11]:
# We now have the basic llm functionality sorted. Can move onto adding agents with CrewAI

<h4>CrewAI

In [12]:
from crewai import Agent, Task, Crew, Process, LLM
from load_keys import *

In [13]:
openai_key_path = Path('keys/open_ai.json')
openai_key = load_openai_key(openai_key_path)
os.environ['OPENAI_API_KEY'] = openai_key

In [14]:
def build_test_crew(llm):
    researcher = Agent(
        role="Researcher",
        backstory="",
        goal="Retrieve and compile evidence using the provided LLM as the sole knowledge source (do NOT call external/local tools).",
        llm=llm,
        tools=[],
        verbose=False,
    )

    writer = Agent(
        role="Content Writer",
        backstory="",
        goal="Write a structured answer grounded in the evidence produced by the Researcher (LLM).",
        llm=llm,
        tools=[],
        verbose=False,
    )

    critic = Agent(
        role="Reviewer",
        backstory="",
        goal="Check the answer for unsupported claims, missing info, and clarity against the Researcher's evidence.",
        llm=llm,
        tools=[],
        verbose=False,
    )

    t1 = Task(
        description=(
            "Research this query using only the LLM: {query}\nReturn bullet findings." # It's important to add the specific variable containing the query hereQ!!
        ),
        expected_output="A bullet list of findings derived from the LLM only.",
        agent=researcher,
    )

    t2 = Task(
        description=(
            "Write the answer. Use the Researcher's findings as the source."
        ),
        expected_output="A Markdown response mapped to the Researcher's findings.",
        agent=writer,
        context=[t1],
    )

    t3 = Task(
        description=(
            "Critique the answer against the Researcher's response. List issues and provide fix instructions."
        ),
        expected_output="A list of issues found plus concrete fix instructions.",
        agent=critic,
        context=[t1, t2],
    )

    t4 = Task(description=(
            "Consider the issues and fix instructions provided by the critic and write a final answer to be outputted to the user"
        ),
        expected_output="A Final answer, written in markdown, which considers the points raised by the critic",
        agent=writer,
        context=[t1, t2, t3],)

    return Crew(
        agents=[researcher, writer, critic],
        tasks=[t1, t2, t3, t4],
        process=Process.sequential,
        verbose=False,
    )

llm_for_crew = LLM(
    model="groq/openai/gpt-oss-120b",
    api_key=os.environ["GROQ_API_KEY"],
    temperature=0.2,
)

test_crew = build_test_crew(llm=llm_for_crew)

In [15]:
# Using the crew:
initial_user_query = "How old is the planet?"

result = test_crew.kickoff(inputs={"query": initial_user_query})
print(result)
# The final answer here mentions the critque mainly...

## How Scientists Determine the Age of Earth – A 4.54 Ga Estimate  

The current scientific consensus is that **Earth is about 4.54 Ga old** (where Ga = billion years), with an overall uncertainty of roughly **± 0.05 Ga** (± 50 million years). This value is **not a direct measurement of Earth itself**; it is **inferred** from the ages of the oldest Solar‑System solids and from a suite of independent chronometers. Below is a structured synthesis of the principal data sets that underpin this age, together with the logical steps that connect them to the final figure.

---

### 1. Radiometric Dating of Primitive Meteorites – The Primary Benchmark  

| Method | Sample | Age (Ga) | Uncertainty (Ga) | Why it matters |
|--------|--------|----------|------------------|----------------|
| **U–Pb (Uranium–Lead)** on **Calcium‑Aluminum‑Rich Inclusions (CAIs)** | Primitive chondritic meteorites (e.g., CV, CO) | **4.567** | **± 0.002** | CAIs are the oldest solid condensates that formed in the solar

╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: 711f30c6-c4dd-4aae-ba5e-ba61223e4d4d                                  │
│                                                                                                                 │
│ 🔗 View here:                                                                                                   │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/711f30c6-c4dd-4aae-ba5e-ba61223e4d4d?access_code=TRA │
│ CE-e3b137a15f                                                                                                   │
│ 🔑 Access Code: TRACE-e3b137a15f                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

<h4>Combining it with the vectorstore

In [16]:
# Writing the local search tool that the CrewAI will use, giving the tool the loaded vecstore:
from typing import Type, List, Dict, Any
from pydantic import BaseModel, Field
from crewai.tools import BaseTool
from langchain_community.vectorstores import FAISS
from docstore_functions import *

In [17]:
# Local search tool for the crew to use:
from pydantic import BaseModel, Field, ConfigDict
from crewai.tools import BaseTool
from langchain_community.vectorstores import FAISS

class LocalSearchArgs(BaseModel):
    query: str = Field(..., description="User question")
    k: int = Field(6, ge=1, le=20, description="Top-k chunks")

class LocalFAISSSearchTool(BaseTool):
    name: str = "local_search"
    description: str = "Search the local FAISS vectorstore and return relevant chunks with source metadata."
    args_schema: Type[BaseModel] = LocalSearchArgs

    model_config = ConfigDict(arbitrary_types_allowed=True)

    store: FAISS

    def _run(self, query: str, k: int = 6) -> List[Dict[str, Any]]:
        docs = self.store.similarity_search(query, k=k)
        out = []
        for i, d in enumerate(docs, start=1):
            out.append({
                "id": f"L-{i:04d}",
                "text": d.page_content,
                "source": d.metadata.get("source", ""),
                "page": d.metadata.get("page", None),
                "content_hash": d.metadata.get("content_hash", None),
            })
        return out
    
# Loading the store:
store,document_list = load_docstore_from_dir()
local_tool = LocalFAISSSearchTool(store=store)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


Loaded FAISS store from 'vectorstore' with 549 document(s).


In [18]:
# Implementing it with a crew:

def build_local_crew(llm, local_tool):
    researcher = Agent(
        role="Researcher",
        backstory="",
        goal="Retrieve and compile evidence from the local FAISS store using the local_search tool.",
        llm=llm,
        tools=[local_tool],
        verbose=False,
    )

    writer = Agent(
        role="Content Writer",
        backstory="",
        goal="Write a structured answer grounded ONLY in the Researcher's retrieved chunks. Cite chunk IDs.",
        llm=llm,
        tools=[],
        verbose=False,
    )

    critic = Agent(
        role="Reviewer",
        backstory="",
        goal="Check the answer for unsupported claims and missing info against the retrieved chunks.",
        llm=llm,
        tools=[],
        verbose=False,
    )

    t1 = Task(
        description=(
            "Query: {query}\n"
            "1) Call local_search(query={query}, k=6).\n"
            "2) Return:\n"
            "- A short 'Evidence' list\n"
            "- Bullet 'Findings' where EACH bullet cites one or more evidence IDs.\n"
        ),
        expected_output="Evidence list + Findings with citations like [L-0001].",
        agent=researcher,
    )

    t2 = Task(
        description=(
            "Write a Markdown answer to: {query}\n"
            "Use ONLY the Evidence/Findings from the Researcher.\n"
            "Every major claim must include citations like [L-0001]."
        ),
        expected_output="Markdown answer with evidence-ID citations.",
        agent=writer,
        context=[t1],
    )

    t3 = Task(
        description=(
            "Critique the Markdown answer for: {query}\n"
            "Flag any claim without a supporting citation from [L-xxxx]. Provide fix instructions."
        ),
        expected_output="Issues + fix instructions.",
        agent=critic,
        context=[t1, t2],
    )

    t4 = Task(description=(
            "Consider the issues and fix instructions provided by the critic and write a final answer to be outputted to the user"
        ),
        expected_output="A Final answer, written in markdown, which considers the points raised by the critic",
        agent=writer,
        context=[t1, t2, t3],)

    return Crew(
        agents=[researcher, writer, critic],
        tasks=[t1, t2, t3,t4],
        process=Process.sequential,
        verbose=False,
    )

In [19]:
local_crew = build_local_crew(llm_for_crew, local_tool)
local_query = "What are the most important reactions used in natural product synthesis?"
local_result = local_crew.kickoff(inputs={"query": local_query})
print(local_result)

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 2758.64it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: BAAI/bge-small-en-v1.5
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


[CrewAIEventsBus] Warning: Event pairing mismatch. 'agent_execution_completed' closed 'llm_call_started' (expected 
'agent_execution_started')

[CrewAIEventsBus] Warning: Event pairing mismatch. 'task_completed' closed 'agent_execution_started' (expected 
'task_started')

[CrewAIEventsBus] Warning: Event pairing mismatch. 'crew_kickoff_completed' closed 'task_started' (expected 
'crew_kickoff_started')

## Most Important Reactions Used in Natural‑Product Synthesis  

| Reaction class | Why it is important in total synthesis | Representative examples |
|----------------|----------------------------------------|--------------------------|
| **Multicomponent reactions (MCRs)** – Mannich, Passerini, Ugi, Biginelli, etc. | Provide rapid, convergent access to highly functionalized intermediates, dramatically shortening synthetic sequences. | MCRs are employed to construct alkaloid cores and peptidomimetic natural products [L‑0001]. |
| **(Intramolecular) Diels‑Alder (IMDA) and tandem Diels‑Alder (TADA) cycloadditions** | Pericyclic reactions that forge multiple C–C bonds and stereocenters in a single step, enabling the construction of polycyclic frameworks typical of terpenoids, alkaloids and polyketides. | IMDA and TADA cyclizations are widely used to build polycyclic terpenoid and alkaloid frameworks [L‑0002, L‑0004]. |
| **Radical‑mediated transformations** – especially tandem intramolec

╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ defb1b24-7294-4382-badc-c6f2a7d98811                                         │
│                                                                              │
│ 🔗 View here:                                                                │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/defb1b24-7294-438 │
│ 2-badc-c6f2a7d98811?access_code=TRACE-d001094035                             │
│ 🔑 Access Code: TRACE-d001094035                                             │
╰──────────────────────────────────────────────────────────────────────────────╯


In [20]:
# We now have the Crew using the local_search. Next step is to implement it with tavily for internet searching.

<h4>tavily (internet search):

In [27]:
# Creating the tavily tool:
from tavily import TavilyClient

In [28]:
class WebSearchArgs(BaseModel):
    query: str = Field(..., description="Search query")
    max_results: int = Field(5, ge=1, le=10, description="Number of results")
    search_depth: str = Field("basic", description="basic or advanced")

class TavilyWebSearchTool(BaseTool):
    name: str = "web_search"
    description: str = "Search the web via Tavily and return relevant snippets with URLs for citation."
    args_schema: Type[BaseModel] = WebSearchArgs

    model_config = ConfigDict(arbitrary_types_allowed=True)

    api_key: str
    client: TavilyClient | None = None

    def model_post_init(self, __context):
        if self.client is None:
            self.client = TavilyClient(api_key=self.api_key)

    def _run(self, query: str, max_results: int = 5, search_depth: str = "basic") -> List[Dict[str, Any]]:
        res = self.client.search(
            query=query,
            max_results=max_results,
            search_depth=search_depth,
        )

        out: List[Dict[str, Any]] = []
        for i, r in enumerate(res.get("results", []), start=1):
            out.append({
                "id": f"W-{i:04d}",
                "title": r.get("title", ""),
                "url": r.get("url", ""),
                "text": r.get("content", "") or r.get("snippet", ""),
                "source": "tavily",
                "score": r.get("score", None),
            })
        return out

In [24]:
# Set tavily API key:
from load_keys import *
p = Path("keys/tavily.json")

In [25]:
tavily_api_key = load_tavily_key(p)
os.environ["TAVILY_API_KEY"] = tavily_api_key

In [30]:
internet_tool = TavilyWebSearchTool(api_key=tavily_api_key)

In [32]:
# Implementing it with a crew:
def build_crew(llm, local_tool,web_tool):
    researcher = Agent(
        role="Researcher",
        backstory="You gather evidence. You MUST use tools. You never answer from memory.",
        goal="Retrieve and compile evidence from local FAISS first, then Tavily web search only if needed.",
        llm=llm,
        tools=[local_tool, web_tool],
        verbose=False,
    )

    writer = Agent(
        role="Content Writer",
        backstory="You write clear answers grounded strictly in evidence provided.",
        goal="Write a structured answer using ONLY the Researcher's evidence. Cite evidence IDs.",
        llm=llm,
        tools=[],
        verbose=False,
    )

    critic = Agent(
        role="Reviewer",
        backstory="You are strict about grounding. No evidence = no claim.",
        goal="Check the draft for unsupported claims, missing coverage, and clarity against the evidence.",
        llm=llm,
        tools=[],
        verbose=False,
    )

    reviser = Agent(
        role="Reviser",
        backstory="You apply reviewer feedback and output the final answer.",
        goal="Revise the draft to fully comply with evidence and reviewer notes. Output final Markdown only.",
        llm=llm,
        tools=[],
        verbose=False,
    )

    t1 = Task(
        description=(
            "User query: {query}\n\n"
            "1) ALWAYS call local_search(query={query}, k=6).\n"
            "2) Decide if web search is necessary:\n"
            "   - If the query asks for recency (latest/today/current/news) OR\n"
            "   - Local evidence is insufficient/weak (few relevant chunks)\n"
            "   then call web_search(query={query}, max_results=5, search_depth='basic').\n\n"
            "Return two sections:\n"
            "A) EVIDENCE (list): each item must include an ID [L-xxxx] or [W-xxxx], plus source/url/page and a 1-sentence summary.\n"
            "B) FINDINGS (bullets): each bullet must cite one or more evidence IDs.\n"
        ),
        expected_output="Evidence list + Findings with citations like [L-0001] and [W-0002].",
        agent=researcher,
    )

    t2 = Task(
        description=(
            "Write a Markdown answer to: {query}\n"
            "Use ONLY the Researcher's EVIDENCE/FINDINGS.\n"
            "Every major claim must have citations like [L-0001] or [W-0002].\n"
            "If evidence is insufficient, say so and suggest what else is needed.\n"
        ),
        expected_output="Markdown answer with evidence-ID citations.",
        agent=writer,
        context=[t1],
    )

    t3 = Task(
        description=(
            "Critique the draft answer for: {query}\n"
            "Check that:\n"
            "- All major claims have citations.\n"
            "- No claim contradicts the evidence.\n"
            "- The answer is complete and clear.\n"
            "Output: (1) Issues list, (2) Concrete fix instructions.\n"
        ),
        expected_output="Issues + fix instructions.",
        agent=critic,
        context=[t1, t2],
    )

    t4 = Task(
        description=(
            "Revise the draft answer for: {query}\n"
            "Apply ALL fix instructions from the Reviewer.\n"
            "Keep citations. Output ONLY the final Markdown answer.\n"
        ),
        expected_output="Final Markdown answer only.",
        agent=reviser,
        context=[t1, t2, t3],
    )

    return Crew(
        agents=[researcher, writer, critic, reviser],
        tasks=[t1, t2, t3, t4],
        process=Process.sequential,
        verbose=False,
    )

In [33]:
internet_tool = TavilyWebSearchTool(api_key=tavily_api_key)
local_tool = LocalFAISSSearchTool(store=store)
overall_crew = build_crew(llm_for_crew,local_tool,internet_tool)

In [40]:
query = "This is a test question. What is today's date?"
result = overall_crew.kickoff(inputs={"query": query})
print(result)

[CrewAIEventsBus] Warning: Event pairing mismatch. 'agent_execution_completed' closed 'llm_call_started' (expected 
'agent_execution_started')

[CrewAIEventsBus] Warning: Event pairing mismatch. 'task_completed' closed 'agent_execution_started' (expected 
'task_started')

[CrewAIEventsBus] Warning: Event pairing mismatch. 'crew_kickoff_completed' closed 'task_started' (expected 
'crew_kickoff_started')

Monday, February 9, 2026【W‑0001】【W‑0002】【W‑0003】


╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ 08c62313-80d5-45e6-b715-1f7afddfe956                                         │
│                                                                              │
│ 🔗 View here:                                                                │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/08c62313-80d5-45e │
│ 6-b715-1f7afddfe956?access_code=TRACE-7c1678a9c7                             │
│ 🔑 Access Code: TRACE-7c1678a9c7                                             │
╰──────────────────────────────────────────────────────────────────────────────╯


In [41]:
query = "List the top news headlines from around the world toady"
result = overall_crew.kickoff(inputs={"query": query})
print(result)

[CrewAIEventsBus] Warning: Event pairing mismatch. 'agent_execution_completed' closed 'llm_call_started' (expected 
'agent_execution_started')

[CrewAIEventsBus] Warning: Event pairing mismatch. 'task_completed' closed 'llm_call_started' (expected 
'task_started')

[CrewAIEventsBus] Warning: Event pairing mismatch. 'crew_kickoff_completed' closed 'llm_call_started' (expected 
'crew_kickoff_started')

## Top World News Headlines (9 February 2026)

- **“Twenty years for Jimmy Lai is a ‘death sentence,’ son says”** – Hong Kong critic Jimmy Lai receives a 20‑year jail term, sparking international outcry【W‑0001】.  
- **“Waymo goes fully autonomous in Nashville”** – Waymo launches a completely driver‑less ride‑hailing service in Nashville, Tennessee【W‑0005】.  
- **“Kroger taps former Walmart exec to lead US grocer, shares rise”** – Kroger appoints a former Walmart executive as CEO, sending its stock higher【W‑0006】.  
- **“Air Canada suspends Cuba flights as island set to run out of jet fuel”** – Air Canada halts all flights to Cuba amid an imminent jet‑fuel shortage on the island【W‑0007】.  
- **“Scenes from Super Bowl LX – Seattle Seahawks beat New England 29‑13”** – The Seattle Seahawks defeat the New England Patriots 29‑13 in Super Bowl LX【W‑0008】.  
- **“Broken medals spark investigation from Milano Games organizers”** – An investigation is launched after several Olympic medals were f

╭────────────────────────── Trace Batch Finalization ──────────────────────────╮
│ ✅ Trace batch finalized with session ID:                                    │
│ deda1bb9-e351-4f65-8d09-6911204ef3e2                                         │
│                                                                              │
│ 🔗 View here:                                                                │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/deda1bb9-e351-4f6 │
│ 5-8d09-6911204ef3e2?access_code=TRACE-aaf0bb5de4                             │
│ 🔑 Access Code: TRACE-aaf0bb5de4                                             │
╰──────────────────────────────────────────────────────────────────────────────╯


In [43]:
# Tavily seems to be working! When we ask for current info (date/news), we get accurate answers.
# Next steps are to extract the classes/functions from this notebook into an importable module script to be used by Streamlit for the frontend...